# Geographic Data
***
Two objectives:
1. Get shapefile data for countries to build a choropleth map
2. Get a mapping of countries to it's continent (to add to the population table)

In [14]:
import pandas as pd
import geopandas as gpd

import requests
import zipfile
import os
import io

# allow web-acces for downloading: https://stackoverflow.com/a/60671292
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from src.data.quick_queries import queryDB
qdb = queryDB('sqlite','../../data/processed/covid_db.sqlite')
%load_ext sql

%load_ext autoreload
%autoreload 2

sqlite:///../../data/processed/covid_db.sqlite
The sql extension is already loaded. To reload it, use:
  %reload_ext sql
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Gather Data
***
Relevant shapefiles for country data can be found on www.naturalearthdata.com. There are three levels of data here (10m, 50m and 110m), where we have chosen for the last option (110m).

This causes us to miss some small countries (i.e. Singapore), however, the other files are found to big to display on interactive choropleth maps.

A potential improvement could be to find the missing countries in the 110m shapfile, and only add more precise (50 or 10m) shapefiles for these, leaving the bigger countries coarse.

#### 1.1. helper function to download & extract zipfiles

In [8]:
def extractZipfile(path, url):
    """
    Extract zipfile from url and store files under path 
    """
    # download
    r = requests.get(url, stream=True)

    with zipfile.ZipFile(io.BytesIO(r.content)) as myzip:
        # get the files inside the zip-file
        file_list = myzip.namelist()
        # extract files one-by-one
        for name in file_list:
            # remove MAXOSX folder
            if name[:8] != '__MACOSX':
                #only keep shapefile
                if name.split('.')[-1] == 'shp':
                    myzip.extract(name, path)
                    return name

#### 1.2.1 Download shapefiles

In [31]:
# extract zipfiles
#url_10 = 'https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_0_countries.zip'
#url_50 = 'https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/cultural/ne_50m_admin_0_countries.zip'
#url_110 = 'https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip'
#path = '../../data/raw'

#filenames = []
#for url in [url_10, url_50, url_110]:
#    filenames.append(extractZipfile(path, url))
#extractZipfile(path, url_10)

#### 1.2.2 Download shapefiles (backup)
Usually, the method outlined in `1.2.1` works. During a check on `13/07/2020` there seemed to be an issue with the regular method for downloading (the zipfiles). We thus deviced the method below to get the data directly from Github.

NB: the .shp file needs additional files (see extensions below) to get the full dataset.

In [43]:
url_10 = 'https://github.com/nvkelso/natural-earth-vector/blob/master/10m_cultural/ne_10m_admin_0_countries.shp?raw=true'
url_50 = 'https://github.com/nvkelso/natural-earth-vector/blob/master/50m_cultural/ne_50m_admin_0_countries.shp?raw=true'
url_110 = 'https://github.com/nvkelso/natural-earth-vector/blob/master/110m_cultural/ne_110m_admin_0_countries.shp?raw=true'
path = '../../data/raw/'

filenames = []
for url in [url_10, url_50, url_110]:
    extensions = ['cpg','dbf','prj','shp','shx']
    for ext in extensions:
        to_download = url.replace('shp', ext)

        # find the filename
        name = to_download.split('/')[-1].split('?')[0]
        if name.split('.')[-1] == 'shp':
            filenames.append(name)

        # extract & store
        r = requests.get(to_download, stream=True)
        with open(path + name, 'wb') as f:
            f.write(r.content)

In [44]:
filenames

['ne_10m_admin_0_countries.shp',
 'ne_50m_admin_0_countries.shp',
 'ne_110m_admin_0_countries.shp']

#### 1.3 Load Geopandas dataframe
We're starting with the coarsest set (110m) and work our way down to 10m, only adding missing countries.

In [45]:
countries_110 = gpd.read_file(path + '/' + filenames[2])[['ADMIN','CONTINENT','geometry']]
print(countries_110.shape)

countries_50 = gpd.read_file(path + '/' + filenames[1])[['ADMIN','CONTINENT','geometry']]
print(countries_50.shape)

countries_10 = gpd.read_file(path + '/' + filenames[0])[['ADMIN','CONTINENT','geometry']]
print(countries_10.shape)

(177, 3)
(241, 3)
(255, 3)


In [46]:
# create a combined dataframe - each geometry as coarse as available
countries = countries_110.copy()
countries = pd.concat([countries, countries_50[~countries_50['ADMIN'].isin(countries['ADMIN'].unique())]], axis=0)
countries = pd.concat([countries, countries_10[~countries_10['ADMIN'].isin(countries['ADMIN'].unique())]], axis=0)
countries.shape

(255, 3)

In [47]:
countries.head()

,ADMIN,CONTINENT,geometry
0,Fiji,Oceania,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,United Republic of Tanzania,Africa,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,Western Sahara,Africa,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,Canada,North America,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,United States of America,North America,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


## 2. Assess Data
***
The goal is to use this data merged with the existing Covid data, again, using the country-name as merging key.

#### 2.1 Compare with countries is `stats` table

In [48]:
# get the countries in our main stats table
query = """
    SELECT DISTINCT country
      FROM stats"""

df = qdb.output_query(query)
df.head(2)

,country
0,Afghanistan
1,Albania


In [49]:
# full join to get comparison
merged = countries.merge(df, left_on = 'ADMIN', right_on = 'country', how = 'outer')
merged.head(2)

,ADMIN,CONTINENT,geometry,country
0,Fiji,Oceania,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",Fiji
1,United Republic of Tanzania,Africa,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",NaN


In [50]:
# not in our shapefile
merged[merged['ADMIN'].isnull()]

,ADMIN,CONTINENT,geometry,country
255,NaN,NaN,None,Bahamas
256,NaN,NaN,None,Congo
257,NaN,NaN,None,Czech Republic
258,NaN,NaN,None,DR Congo
259,NaN,NaN,None,Eswatini
260,NaN,NaN,None,Holy See
261,NaN,NaN,None,North Macedonia
262,NaN,NaN,None,Sao Tome and Principe
263,NaN,NaN,None,Serbia
264,NaN,NaN,None,State of Palestine


In [51]:
# not in our stats table - mostly remove
merged[merged['country'].isnull()]

,ADMIN,CONTINENT,geometry,country
1,United Republic of Tanzania,Africa,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",NaN
4,United States of America,North America,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",NaN
11,Democratic Republic of the Congo,Africa,"POLYGON ((29.34000 -4.49998, 29.51999 -5.41998...",NaN
19,The Bahamas,North America,"MULTIPOLYGON (((-78.98000 26.79000, -78.51000 ...",NaN
20,Falkland Islands,South America,"POLYGON ((-61.20000 -51.85000, -60.00000 -51.2...",NaN
...,...,...,...,...
250,Spratly Islands,Asia,"MULTIPOLYGON (((115.36720 10.23749, 115.36598 ...",NaN
251,Clipperton Island,Seven seas (open ocean),"POLYGON ((-109.21203 10.30268, -109.21036 10.2...",NaN
252,Bajo Nuevo Bank (Petrel Is.),North America,"POLYGON ((-79.98929 15.79495, -79.98782 15.796...",NaN
253,Serranilla Bank,North America,"POLYGON ((-78.63707 15.86209, -78.64041 15.864...",NaN


## 3. Clean Data
***
Similar to the process when we merged our main Covid stats with population, we'll have to manually clean some country names. 

In [52]:
# dictionary to capture our translation [old to new]
country_translation = {
    'United Republic of Tanzania' : 'Tanzania',
    'Republic of the Congo' : 'Congo',
    'Democratic Republic of the Congo' : 'DR Congo',
    'Palestine' : 'State of Palestine',
    'eSwatini' : 'Eswatini',
    'The Bahamas' : 'Bahamas',
    'Czechia' : 'Czech Republic',
    'Macedonia' : 'North Macedonia',
    'São Tomé and Principe' : 'Sao Tome and Principe',
    'Republic of Serbia' : 'Serbia',
    'United States of America' : 'United States',
    'East Timor' : 'Timor-Leste',
    'Vatican' : 'Holy See'}

# update
countries['ADMIN'].replace(country_translation, inplace = True)
countries.reset_index(drop=True, inplace=True)

In [53]:
# check
merged2 = countries.merge(df, left_on = 'ADMIN', right_on = 'country', how = 'outer')
merged2[merged2['ADMIN'].isnull()]

,ADMIN,CONTINENT,geometry,country


In [54]:
# rename columns
countries.columns = ['country','continent','geometry']
countries.head()

,country,continent,geometry
0,Fiji,Oceania,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,Tanzania,Africa,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,Western Sahara,Africa,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,Canada,North America,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,United States,North America,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


## 4. Store Data
***
We're going to try to add the continent at least to the `populations` table. 

There are extensions to add geometry as well to a sqlite DB, however we deem this outside the scope of this assignment. We're storing this data as shapefile for later use.

#### 4.1 add continent to populations table

In [55]:
# update table with new column
query = """
    ALTER TABLE populations
    ADD continent varchar"""

qdb.admin_query(query)

In [56]:
# insert into this column
for i in range(len(countries)):
    if countries['country'][i] in list(df.country):
        query = """
            UPDATE populations
            SET continent = '{}'
            WHERE country = '{}'
            """.format(countries['continent'][i], countries['country'][i])

        qdb.admin_query(query)

In [57]:
%%sql sqlite:///../../data/processed/covid_db.sqlite
SELECT *
  FROM populations
 LIMIT 5;

Done.


rank,country,population,yearly_change_pct,net_change,density,land_are,migrants,fert_rate,med_age,urban_pop_pct,world_share_pct,continent
1,China,1439323776,0.39,5540090,153,9388211,-348399,1.7,38,61,18.47,Asia
2,India,1380004385,0.99,13586631,464,2973190,-532687,2.2,28,35,17.7,Asia
3,United States,331002651,0.59,1937734,36,9147420,954806,1.8,38,83,4.25,North America
4,Indonesia,273523615,1.07,2898047,151,1811570,-98955,2.3,30,56,3.51,Asia
5,Pakistan,220892340,2,4327022,287,770880,-233379,3.6,23,35,2.83,Asia


#### 4.2 Store geometric data

In [58]:
countries.to_file("../../data/processed/countries.shp")

## 5. Late fix
***
We found that some of our countries are mapped to the continent `Seven seas (open ocean)'

#### 5.1 Issue: incorrect continent

In [59]:
%%sql
SELECT DISTINCT
    continent, country
FROM populations
WHERE continent = 'Seven seas (open ocean)'
LIMIT 5;

 * sqlite:///../../data/processed/covid_db.sqlite
Done.


continent,country
Seven seas (open ocean),Mauritius
Seven seas (open ocean),Maldives
Seven seas (open ocean),Seychelles


#### 5.2 fix

In [60]:
# country + correct continent
map_continent = {'Seychelles' : 'Africa',
                 'Mauritius' : 'Africa',
                 'Maldives' : 'Asia'}

for key in list(map_continent.keys()):
    query = """
            UPDATE populations
            SET continent = '{}'
            WHERE country = '{}'
            """.format(map_continent[key], key)
    #print(query)
    qdb.admin_query(query)

#### 5.3 check

In [61]:
%%sql
SELECT DISTINCT
    continent, country
FROM populations
WHERE continent = 'Seven seas (open ocean)'
LIMIT 5;

 * sqlite:///../../data/processed/covid_db.sqlite
Done.


continent,country


In [62]:
%%sql
SELECT DISTINCT continent
  FROM populations
       JOIN stats 
         ON stats.country = populations.country;

 * sqlite:///../../data/processed/covid_db.sqlite
Done.


continent
Asia
North America
South America
Africa
Europe
Oceania
